In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🌀 02 - CT Reconstruction Baselines\n",
    "\n",
    "This notebook explores classical CT reconstruction methods:\n",
    "- Simulated parallel-beam sinograms\n",
    "- Filtered Back Projection (FBP)\n",
    "- Simultaneous Iterative Reconstruction Technique (SIRT)\n",
    "- Evaluation using PSNR and SSIM"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim\n",
    "from skimage.data import shepp_logan_phantom\n",
    "from skimage.transform import radon, iradon\n",
    "import astra\n",
    "import astra.toolbox.sirt as sirt\n",
    "\n",
    "plt.style.use('seaborn')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🏗️ Load Ground Truth (2D Shepp-Logan Phantom)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "image = shepp_logan_phantom()\n",
    "image = np.pad(image, ((28, 28), (28, 28)), mode='constant')  # 256x256\n",
    "plt.imshow(image, cmap='gray')\n",
    "plt.title(\"Ground Truth Phantom\")\n",
    "plt.axis('off')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📐 Simulate Sinogram (Radon Transform)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "theta = np.linspace(0., 180., max(image.shape), endpoint=False)\n",
    "sinogram = radon(image, theta=theta)\n",
    "plt.imshow(sinogram, cmap='gray', aspect='auto')\n",
    "plt.title(\"Simulated Sinogram\")\n",
    "plt.xlabel(\"Projection angle\")\n",
    "plt.ylabel(\"Detector\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🌀 Filtered Back Projection (FBP)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fbp_recon = iradon(sinogram, theta=theta, circle=True)\n",
    "plt.imshow(fbp_recon, cmap='gray')\n",
    "plt.title(\"FBP Reconstruction\")\n",
    "plt.axis('off')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔁 SIRT Reconstruction (ASTRA)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup ASTRA geometry\n",
    "vol_geom = astra.create_vol_geom(image.shape[0], image.shape[1])\n",
    "proj_geom = astra.create_proj_geom('parallel', 1.0, image.shape[0], theta)\n",
    "\n",
    "# Create sinogram in ASTRA format\n",
    "sinogram_id = astra.data2d.create('-sino', proj_geom, sinogram)\n",
    "rec_id = astra.data2d.create('-vol', vol_geom)\n",
    "\n",
    "# Create and run algorithm\n",
    "cfg = astra.astra_dict('SIRT')\n",
    "cfg['ProjectionDataId'] = sinogram_id\n",
    "cfg['ReconstructionDataId'] = rec_id\n",
    "alg_id = astra.algorithm.create(cfg)\n",
    "astra.algorithm.run(alg_id, 50)\n",
    "\n",
    "sirt_recon = astra.data2d.get(rec_id)\n",
    "astra.algorithm.delete(alg_id)\n",
    "\n",
    "# Display\n",
    "plt.imshow(sirt_recon, cmap='gray')\n",
    "plt.title(\"SIRT Reconstruction (50 iterations)\")\n",
    "plt.axis('off')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Reconstruction Quality Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fbp_psnr = psnr(image, fbp_recon)\n",
    "fbp_ssim = ssim(image, fbp_recon)\n",
    "\n",
    "sirt_psnr = psnr(image, sirt_recon)\n",
    "sirt_ssim = ssim(image, sirt_recon)\n",
    "\n",
    "print(\"FBP - PSNR: {:.2f}, SSIM: {:.4f}\".format(fbp_psnr, fbp_ssim))\n",
    "print(\"SIRT - PSNR: {:.2f}, SSIM: {:.4f}\".format(sirt_psnr, sirt_ssim))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ✅ Summary\n",
    "\n",
    "- FBP is fast and provides reasonable results with noise.\n",
    "- SIRT is slower but more accurate for low-dose or sparse data.\n",
    "- These serve as baselines before GAN-based reconstructions.\n",
    "\n",
    "**Next**: Move to `03_gan_artifact_removal.ipynb` for learning-based improvements."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": ""
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
